In [1]:
import pandas as pd
import numpy as np
from requests import Request, Session
import requests
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
from bs4 import BeautifulSoup
from apscheduler.schedulers.background import BackgroundScheduler
import datetime
import pytz

# Create class to store/append data
class dai_data:
    def __init__(self):
        self.data = pd.DataFrame()

    def update_data(self):
        new_entry = get_dex_data()
        self.data = pd.concat([self.data, new_entry], axis = 0).reset_index(drop=True)
        self.data.to_csv('dex_data.csv')

# Initialize data object
dai = dai_data()

# API call
def get_dex_data():
    dai_data = pd.DataFrame()
    for trans in ['buy','sell']:
        
        for amount in [10000, 25000, 50000, 100000, 200000, 500000, 1000000, 1500000]:
            results = {}
            if trans == 'buy':
                results['buy_or_sell'] = 'buy'
                dex_URL = f"https://api-v2.dex.ag/price?from=USDC&to=DAI&fromAmount={amount}&dex=ag"
                # page = requests.get(dex_URL)
                # dex = json.loads(page.text)
                dex = requests.get(dex_URL).json()

                results['amount'] = amount
                results['price'] = 1 / float(dex['price'])

                # add liquidity
                for item in dex['liquidity']:
                    results['liquidity_' + item] = dex['liquidity'][item]

                # timestamp
                results['date'] = datetime.datetime.now(pytz.timezone('America/Cayman')).date()
                results['time'] = datetime.datetime.now(pytz.timezone('America/Cayman')).time()
            else:
                results['buy_or_sell'] = 'sell'
                dex_URL = f"https://api-v2.dex.ag/price?from=DAI&to=USDC&fromAmount={amount}&dex=ag"
                page = requests.get(dex_URL)
                dex = json.loads(page.text)
                
                results['amount'] = amount
                results['price'] = float(dex['price'])

                # add liquidity
                for item in dex['liquidity']:
                    results['liquidity_' + item] = dex['liquidity'][item]

                # timestamp
                results['date'] = datetime.datetime.now(pytz.timezone('America/Cayman')).date()
                results['time'] = datetime.datetime.now(pytz.timezone('America/Cayman')).time()  
           
            results = pd.DataFrame([results])
            dai_data = pd.concat([dai_data, results])

    return dai_data

   

# # API call
# def get_coingecko_data(metrics):
#     '''
#     metrics = list of strings, ex. ['symbol', 'market_cap', 'total_volume']
#     '''
#     # Call coin gecko api
#     CG_URL = "https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&ids=dai&order=market_cap_desc&per_page=100&page=1&sparkline=false"
#     page = requests.get(CG_URL) 
#     coin_gecko = json.loads(page.text)[0]

#     # Get desired metrics
#     results = {}
#     for i in metrics:
#         results[i] = coin_gecko[i]
#     return results

# # API call
# def get_dex_data():
#     results = {}
#     # access dex api
#     dex_URL = f"https://api-v2.dex.ag/price?from=DAI&to=USDC&fromAmount=400000&dex=ag"
#     page = requests.get(dex_URL)
#     dex = json.loads(page.text)
    
#     # add price to dai series
#     results['price'] = dex['price']

#     # add liquidity
#     for item in dex['liquidity']:
#         results['liquidity_' + item] = dex['liquidity'][item]
#     return results

# # Combine API calls, timestamp
# def add_data_entry():
#     #coin_gecko_data = get_coingecko_data(metrics = ['market_cap', 'total_volume', 'market_cap_change_24h', 'market_cap_change_percentage_24h','circulating_supply'])
#     dex_data = get_dex_data()

#     # Merge dicts
#     #merged = {**coin_gecko_data, **dex_data}

#     # timestamp
#     merged['date'] = datetime.datetime.now().date()
#     merged['time'] = datetime.datetime.now().time()

#     return pd.DataFrame([merged])



In [2]:
import schedule

schedule.every().minute.do(dai.update_data)

while True:
    schedule.run_pending()

KeyboardInterrupt: 

In [83]:

# Access coin gecko api
CG_URL = "https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&ids=dai%2C%20ethereum&order=market_cap_desc&per_page=100&page=1&sparkline=false"
page = requests.get(CG_URL) 
coin_gecko = json.loads(page.text)

In [84]:
# Create empty pd.series
new_entry = pd.Series()
desired_metrics = ['symbol', 'last_updated', 'market_cap', 'market_cap_rank', 'total_volume', 'market_cap_change_24h', 'market_cap_change_percentage_24h','circulating_supply']

# Get desired metrics
for coin in coin_gecko.text:
    for i in desired_metrics:
        new_entry[coin['symbol'] + '_' + i] = coin[i]

TypeError: string indices must be integers